# Multi-Player Dungeons & Dragons

This notebook shows how the `DialogueAgent` and `DialogueSimulator` class make it easy to extend the [Two-Player Dungeons & Dragons example](https://python.langchain.com/en/latest/use_cases/agent_simulations/two_player_dnd.html) to multiple players.

The main difference between simulating two players and multiple players is in revising the schedule for when each agent speaks

To this end, we augment `DialogueSimulator` to take in a custom function that determines the schedule of which agent speaks. In the example below, each character speaks in round-robin fashion, with the storyteller interleaved between each player.

In [1]:
%load_ext autoreload
%autoreload 2
%autosave 60

Autosaving every 60 seconds


In [2]:
import os
#load from json .creds/PINECONE_API
import json
with open('.creds/PINECONE_API') as f:
    creds = json.load(f)
    PINECONE_API_KEY = creds['PINECONE_API_KEY']
    PINECONE_ENVIRONMENT = creds['PINECONE_ENVIRONMENT']
    OPENAI_API_KEY = creds['OPENAI_API_KEY']

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

# Similarity search in Pinecone

Constrains in location, age, image like, etc

In [4]:
from src.mock_database import pinecone_example, similarity_search
user_a, user_b = similarity_search(pinecone_example)

In [5]:
print("User A:", user_a['name'])
print("User B:", user_b['name'])

User A: John Sullivan
User B: Mary Smith


In [6]:
from src.agent_type import DialogueAgent
from src.simulation_type import DialogueSimulator

## Define roles and quest

In [7]:
character_names = [user_a['name'], user_b['name']]
storyteller_name = "God"
quest = "Find the defects and positive values of the other person. Propose life plans and evaluate if they are compatible. Determine what you like and what you don't like about the other."
word_limit = 50  # word limit for task brainstorming

## Detail to date conditions

In [8]:
game_description = f"""Here is the topic for a Love & Hate game: {quest}.
        The characters are: {*character_names,}.
        The conditions of the date is narrated by, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Dungeons & Dragons player."
)


def generate_character_description(character_name):
    """TODO: Re think this to take the key value pairs from the database and lookup the description"""
    if character_name == user_a["name"]:
        character_description = user_a["description"]
    else:
        character_description = user_b["description"]
    return character_description


def generate_character_system_message(character_name, character_description):
    return SystemMessage(
        content=(
            f"""{game_description}
    Your name is {character_name}. 
    Your character description is as follows: {character_description}.
    You will propose actions you plan to take and {storyteller_name} will explain what happens when you take those actions.
    Speak in the first person from the perspective of {character_name}.
    For describing your own body movements, wrap your description in '*'.
    Do not change roles!
    Do not speak from the perspective of anyone else.
    Remember you are {character_name}.
    Stop speaking the moment you finish speaking from your perspective.
    Never forget to keep your response to {word_limit} words!
    Do not add anything else.
    """
        )
    )


character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_system_messages = [
    generate_character_system_message(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
         The objective is to have a god that acts to generate conflicts or positive acts in the enviroment of the date. Please reply with a creative description of the storyteller. {storyteller_name}, in {word_limit} words or less.
        Speak directly to {storyteller_name}.
        Do not add anything else."""
    ),
]
storyteller_description = ChatOpenAI(temperature=1.0)(
    storyteller_specifier_prompt
).content

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
You are the storyteller, {storyteller_name}. 
Your description is as follows: {storyteller_description}.
The other players will propose actions to take and you will explain what happens when they take those actions.
Speak in the first person from the perspective of {storyteller_name}.
Do not change roles!
Do not speak from the perspective of anyone else.
Remember you are the storyteller, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
"""
    )
)

In [9]:
print("Storyteller Description:")
print(storyteller_description)
for character_name, character_description in zip(
    character_names, character_descriptions
):
    print(f"{character_name} Description:")
    print(character_description)

Storyteller Description:
Dear God, please narrate the conditions of the date for John Sullivan and Mary Smith. Please use your divine powers to generate conflicts or positive acts in their environment. As the storyteller, you have the power to create the perfect atmosphere for them.
John Sullivan Description:
Hey there! I'm an athletic and sports-loving guy who's passionate about staying active and living a healthy lifestyle. I find great joy in the thrill of sports, whether it's playing soccer, hitting the basketball court, or going for a long run in nature. I'm looking to connect with someone who shares my enthusiasm for sports and adventure. Whether you're an athlete yourself or simply enjoy being active, I believe that shared passions can be the foundation of a strong connection. When I'm not on the field or in the gym, I enjoy exploring new hiking trails, catching live sporting events, and even trying out different cuisines to fuel my active lifestyle. I value spontaneity, laughte

## Use an LLM to create an elaborate quest description

In [10]:
quest_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the characters: {*character_names,}.
        Do not add anything else."""
    ),
]
specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

Original quest:
Find the defects and positive values of the other person. Propose life plans and evaluate if they are compatible. Determine what you like and what you don't like about the other.

Detailed quest:
John and Mary, determine five qualities you appreciate and five you dislike in each other. Propose a dream life together and evaluate if it aligns with your goals. Then, swap roles and argue the opposite stance. Can you still find common ground? Time is ticking! Will you end up together or apart?



## Main Loop

In [11]:
characters = []
for character_name, character_system_message in zip(
    character_names, character_system_messages
):
    characters.append(
        DialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
        )
    )
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=ChatOpenAI(temperature=0.2),
)

In [12]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    """
    If the step is even, then select the storyteller
    Otherwise, select the other characters in a round-robin fashion.

    For example, with three characters with indices: 1 2 3
    The storyteller is index 0.
    Then the selected index will be as follows:

    step: 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16

    idx:  0  1  0  2  0  3  0  1  0  2  0  3  0  1  0  2  0
    """
    if step % 2 == 0:
        idx = 0
    else:
        idx = (step // 2) % (len(agents) - 1) + 1
    return idx

In [13]:
max_iters = 20
n = 0

simulator = DialogueSimulator(
    agents=[storyteller] + characters, selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(God): John and Mary, determine five qualities you appreciate and five you dislike in each other. Propose a dream life together and evaluate if it aligns with your goals. Then, swap roles and argue the opposite stance. Can you still find common ground? Time is ticking! Will you end up together or apart?


(John Sullivan): I appreciate Mary's intelligence, sense of humor, kindness, adventurous spirit, and passion for learning. However, I dislike her lack of interest in sports, tendency to procrastinate, indecisiveness, pessimism, and disorganization. In terms of a dream life together, I envision us traveling the world, trying new sports and activities together, and supporting each other's personal and professional goals. I see us living in a cozy home with a big backyard where we can host barbecues and play sports with friends and family. However, I'm not sure if Mary's career aspirations align with my desire for flexibility and travel. We'll have to discuss and compromise to make it wo